1) IMPORTAZIONE LIBRERIE E CARICAMENTO DATI

In [154]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [155]:
CSV_files ={"customers": r"dataset/olist_customers_dataset.csv",
"geolocation_dataset":r"dataset/olist_geolocation_dataset.csv",
"order_items":r"dataset/olist_order_items_dataset.csv",
"order_payments":r"dataset/olist_order_payments_dataset.csv",
"order_review":r"dataset/olist_order_reviews_dataset.csv",
"order_dataset":r"dataset/olist_orders_dataset.csv",
"list_product":r"dataset/olist_products_dataset.csv",
"list_seller":r"dataset/olist_sellers_dataset.csv",
"product_category":r"dataset/product_category_name_translation.csv"}

In [156]:
dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

In [216]:
df_customers = dataframes["customers"]
df_orders_items = dataframes["order_items"]
df_order_payments = dataframes["order_payments"]
df_order_review= dataframes["order_review"]
df_order_dataset = dataframes["order_dataset"]
df_list_product = dataframes["list_product"]
df_list_seller = dataframes["list_seller"]
df_product_category = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

2) ANALISI PRELIMINARE DEI DATI CARICATI

In [218]:
df_customers.info()
df_customers.sample(2)



<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   customer_id               99441 non-null  str  
 1   customer_unique_id        99441 non-null  str  
 2   customer_zip_code_prefix  99441 non-null  int64
 3   customer_city             99441 non-null  str  
 4   customer_state            99441 non-null  str  
dtypes: int64(1), str(4)
memory usage: 3.8 MB


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
90696,7c38e0023219a6589c20a892d51c5598,6567ecb9d5b5a37f19bbeb5fb1972fef,11015,santos,SP
98043,3eeafe2383280a4ff135e5c06965c18b,ecef47a5c983479795ed56b3b6ae1702,82630,curitiba,PR


In [158]:
df_orders_items.info()
df_orders_items.sample(3)


<class 'pandas.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  str    
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  str    
 3   seller_id            112650 non-null  str    
 4   shipping_limit_date  112650 non-null  str    
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), str(4)
memory usage: 6.0 MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
110646,fb6a01b94b2f20833eb7593ea1a54a8f,1,88774687a8883c2c512e37de090bc725,ceaec5548eefc6e23e6607c5435102e7,2018-02-28 20:46:57,389.0,16.33
14504,20fd43d1a1e8d1257044b8a17f3c7c5e,2,7bbc646134de228a74392d8407d16df8,ddd51ae8cda92f3995a51fc0f0f3eec7,2017-04-10 01:50:16,34.9,9.34
43703,63576020815760d4b5d11cef32595ada,1,53b36df67ebb7c41585e8d54d6772e08,4869f7a5dfa277a7dca6462dcf3b52b2,2017-09-08 18:35:29,169.9,5.82


In [159]:
df_order_payments.info()
df_order_payments.sample(3)

<class 'pandas.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  str    
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  str    
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), str(2)
memory usage: 4.0 MB


,order_id,payment_sequential,payment_type,payment_installments,payment_value
44099,79fe183c2c4f23688615d54004cd3f2d,1,credit_card,1,79.51
610,816b15129b14c066ff4f5755938aefb0,1,boleto,1,126.44
20418,86af9599ffd64a39d743904daa8bdb2d,1,credit_card,1,36.84


In [160]:
df_order_review.info()
df_order_review.sample(3)

<class 'pandas.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   review_id                99224 non-null  str  
 1   order_id                 99224 non-null  str  
 2   review_score             99224 non-null  int64
 3   review_comment_title     11568 non-null  str  
 4   review_comment_message   40977 non-null  str  
 5   review_creation_date     99224 non-null  str  
 6   review_answer_timestamp  99224 non-null  str  
dtypes: int64(1), str(6)
memory usage: 5.3 MB


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
26893,ea8eb55610fffc65f03c5f58fa3474f5,4c33f92b62a822db77b38b88bc7ad237,4,NaN,Ainda não recebi. Os correios não atualiza o s...,2018-03-14 00:00:00,2018-03-19 13:37:30
30566,57e6cffda3be027a7c2d3b1661d3b31b,c05b4afd0213a3d8f5c4c503b36dfb87,5,NaN,NaN,2018-03-24 00:00:00,2018-03-30 03:01:32
16727,d333e1c050d143b9e7fe46b56cdf3dbd,25f4849a6afc2184f9c368d736e18341,5,NaN,NaN,2018-05-01 00:00:00,2018-05-02 09:33:07


In [161]:
df_order_dataset.info()
df_order_dataset.sample(3)


<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   order_id                       99441 non-null  str  
 1   customer_id                    99441 non-null  str  
 2   order_status                   99441 non-null  str  
 3   order_purchase_timestamp       99441 non-null  str  
 4   order_approved_at              99281 non-null  str  
 5   order_delivered_carrier_date   97658 non-null  str  
 6   order_delivered_customer_date  96476 non-null  str  
 7   order_estimated_delivery_date  99441 non-null  str  
dtypes: str(8)
memory usage: 6.1 MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
60513,40fd2a55eacc12019cc034c4cc15674c,0c3f432e2cf7ebb3a2cd874a8da89541,delivered,2018-06-04 00:24:10,2018-06-04 00:31:24,2018-06-05 13:52:00,2018-06-22 21:41:53,2018-07-13 00:00:00
12039,004ba47bb589cdbc485e6025bd6d3c81,ab30bf72cae845f3814c7baa50ca549a,delivered,2017-12-25 20:40:15,2017-12-25 20:48:10,2017-12-26 20:39:05,2017-12-29 20:24:23,2018-01-19 00:00:00
33453,96a37e69d0e47674bb4214448cf76dc8,4fd779435ce96ecf52189efb9ee329de,delivered,2017-09-01 13:44:38,2017-09-01 13:55:14,2017-09-12 02:15:52,2017-09-13 14:28:01,2017-09-20 00:00:00


In [162]:
df_list_product.info()
df_list_product.sample(3)

<class 'pandas.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  str    
 1   product_category_name       32341 non-null  str    
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.3 MB


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
12751,ea7372593696ba29709de6d6d3092f48,moveis_decoracao,34.0,1918.0,3.0,8100.0,80.0,4.0,55.0
4703,6ddcabeba730cb81a3c9570a595a12da,beleza_saude,53.0,386.0,1.0,700.0,30.0,12.0,16.0
17395,e47eff9f97c9536c9f768f86e13252f6,NaN,NaN,NaN,NaN,10250.0,40.0,32.0,32.0


In [163]:
df_list_seller.info()
df_list_seller.sample(3)

<class 'pandas.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   seller_id               3095 non-null   str  
 1   seller_zip_code_prefix  3095 non-null   int64
 2   seller_city             3095 non-null   str  
 3   seller_state            3095 non-null   str  
dtypes: int64(1), str(3)
memory usage: 96.8 KB


,seller_id,seller_zip_code_prefix,seller_city,seller_state
1028,9539213aa8e023c01937bef95dc006da,3673,sao paulo,SP
1023,ed859002ad59dbf8cf3602696a6c3000,86035,londrina,PR
904,dd88d13bea9e4c2bf47daf224f8141e5,95900,lajeado,RS


In [164]:
df_product_category.info()
df_product_category.sample(3)

<class 'pandas.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   product_category_name          71 non-null     str  
 1   product_category_name_english  71 non-null     str  
dtypes: str(2)
memory usage: 1.2 KB


,product_category_name,product_category_name_english
33,eletrodomesticos,home_appliances
15,telefonia_fixa,fixed_telephony
65,dvds_blu_ray,dvds_blu_ray


In [165]:
df_geolocation_dataset.info()
df_geolocation_dataset.sample(3)


<class 'pandas.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  str    
 4   geolocation_state            1000163 non-null  str    
dtypes: float64(2), int64(1), str(2)
memory usage: 38.2 MB


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
686305,44600,-12.159186,-39.740581,ipira,BA
664622,39710,-18.616457,-42.279558,coroaci,MG
792793,73040,-15.648355,-47.783399,brasilia,DF


3. PULIZIA PRELIMINARE DEI DATAFRAME (list_product e merge delo stesso con ctagory name)

In [166]:
df_list_product.info()#individuati 610 records che non riportano la categoria prodotto

<class 'pandas.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  str    
 1   product_category_name       32341 non-null  str    
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.3 MB


In [167]:
df_list_product=df_list_product.drop(['product_name_lenght','product_width_cm','product_height_cm','product_length_cm','product_weight_g'],axis=1)
#cancellazione delle colonne relative alle dimensioni(lunghezza del nome,largfhezza,altezza, lunghezza, larghezzza)
#LASCIATO QUANTITA' FOTO E LUNGHEZZA DESCRIZIONE (ELEMENTI PER UN ANALISI?? PIù VENDUTI CON PIù FOTO PIù VENDUTI CON PIù DESCRIZIONE)


In [168]:
product_id_nan=df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete=product_id_nan['product_id']
product_id_to_delete=product_id_to_delete.to_numpy()
#creato un array con gli id prodototto che non hanno categorie per usarlo come criteri di esclusione negli altri df

In [169]:
df_list_product=df_list_product.dropna()
df_list_product.info()#pulito il df  da i valori nulli probabilemnte ho perso 1 id product 32340 invece di  32341

<class 'pandas.DataFrame'>
Index: 32341 entries, 0 to 32950
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32341 non-null  str    
 1   product_category_name       32341 non-null  str    
 2   product_description_lenght  32341 non-null  float64
 3   product_photos_qty          32341 non-null  float64
dtypes: float64(2), str(2)
memory usage: 1.2 MB


In [170]:
df_list_product=df_list_product.merge(df_product_category,on='product_category_name')
#AGGREGATA LA COLONNA CON IL NOME I INLGESE DELLA CATEGORIA PRODOTTO

In [171]:
df_list_product=df_list_product.drop(['product_category_name'],axis=1)

In [172]:
df_list_product.sample(2)#pulito il database-lista prodotti e aggragte callse categoria prodotto in inglese

,product_id,product_description_lenght,product_photos_qty,product_category_name_english
2323,f5d9f3f39171645ad59c424570874afb,967.0,1.0,health_beauty
25892,ad4b5def91ac7c575dbdf65b5be311f4,665.0,1.0,computers_accessories


4) PULIZIA PRELIMINARE DATAFRAME df_orders_items e cambio formato to datetime

In [173]:
df_orders_items_clean=df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
df_orders_items_clean.info()#creato il dataframe df_orders ripulito dai prodotti senza categoria

<class 'pandas.DataFrame'>
Index: 111047 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             111047 non-null  str    
 1   order_item_id        111047 non-null  int64  
 2   product_id           111047 non-null  str    
 3   seller_id            111047 non-null  str    
 4   shipping_limit_date  111047 non-null  str    
 5   price                111047 non-null  float64
 6   freight_value        111047 non-null  float64
dtypes: float64(2), int64(1), str(4)
memory usage: 6.8 MB


In [174]:
df_orders_items_clean['shipping_limit_date']=pd.to_datetime(df_orders_items_clean['shipping_limit_date'])
#convertito in data la colonna di shipping date time

In [182]:
df_orders_items_clean.sample(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
36712,535981e0e0cb431bfd3a85a4279f03d9,1,aa0d48704d551d1247be98ca06dfa990,d4e12e7884759a14fa0f5f896c791cae,2017-10-19 13:49:24,49.8,14.10
31595,47aa694d87f6ae4995bd02d6e8d9132d,1,b17a6664cea6ac6c23ff388f5a67c973,7040e82f899a04d1b434b795a43b4617,2018-04-03 23:15:12,24.9,8.29


5) PULIZA PRELIMINARE DATAFRAME geolocation E RAGGRUPAMENTO PER ZIP CODE

In [175]:
df_geolocation_clean.sample(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
43570,3022,-23.524591,-46.605241,sao paulo,SP
733939,58807,-6.762441,-38.234228,sousa,PB
340126,14701,-20.956453,-48.482396,bebedouro,SP


In [176]:
brazil= {
    "lat_min": -34.0, "lat_max": 5.0,
    "lng_min": -75.0, "lng_max": -28.0
}
df_geolocation_clean = df_geolocation_dataset[df_geolocation_dataset["geolocation_lat"] >= brazil["lat_min"]]
df_geolocation_clean = df_geolocation_clean[df_geolocation_clean["geolocation_lat"] <= brazil["lat_max"]]
df_geolocation_clean = df_geolocation_clean[df_geolocation_clean["geolocation_lng"] >= brazil["lng_min"]]
df_geolocation_clean = df_geolocation_clean[df_geolocation_clean["geolocation_lng"] <= brazil["lng_max"]]
df_geolocation_dataset=df_geolocation_clean     #limitato i valori di lat e lng in brazil
df_geolocation_dataset=df_geolocation_dataset.groupby('geolocation_zip_code_prefix').aggregate({
    'geolocation_lat': 'mean', 'geolocation_lng': 'mean', 'geolocation_city': 'first', 'geolocation_state': 'first'})  #raggruppato per zip code
df_geolocation_dataset.info()
#filtrate le coordinate dentro il Brasile il nome dello stato è spesso scritto in maniera diversa

<class 'pandas.DataFrame'>
Index: 19011 entries, 1001 to 99990
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   geolocation_lat    19011 non-null  float64
 1   geolocation_lng    19011 non-null  float64
 2   geolocation_city   19011 non-null  str    
 3   geolocation_state  19011 non-null  str    
dtypes: float64(2), str(2)
memory usage: 742.6 KB


In [177]:
df_geolocation_dataset.sample(3)

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
38048,-19.770179,-47.944846,uberaba,MG
74765,-16.648370,-49.220738,goiania,GO
9691,-23.669856,-46.589829,sao bernardo do campo,SP


In [178]:
df_list_seller=df_list_seller.rename(columns={'seller_zip_code_prefix':'geolocation_zip_code_prefix'})

In [215]:
df_list_seller.sample(3)

,seller_id,geolocation_zip_code_prefix,seller_city,seller_state
2132,443d880f15cbd3572885e1d44bf2c478,17506,marilia,SP
102,422be4cc81a457fdb46f47edeb968ae5,14940,ibitinga,SP
1412,6179a28a13a726c29b3bf54c070dccab,37048,varginha,MG


In [ ]:
df_customers=df_customers.rename(columns={'customer_zip_code_prefix':'geolocation_zip_code_prefix'})

In [222]:
df_customers.sample(3)

,customer_id,customer_unique_id,geolocation_zip_code_prefix,customer_city,customer_state
57857,d608d88b8e4dfd4114634d9759b2f63b,8672605892b455cfa79b6f69ca37d153,36800,carangola,MG
27961,d9a4e05b8c763933f3a6c9640d64a75d,7fb793be0a754449f7f3dd2e09d695ff,28613,nova friburgo,RJ
34714,16c0be52bf7bf9069c6fa0d850c660ac,88b88849a236b3f007be551d5c10daf8,3638,sao paulo,SP
